# Importing Packages


In [1]:
import sklearn
import matplotlib.pyplot as plt
from sklearn import svm
import sklearn.model_selection as model_selection

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set(font_scale = 1.2)
%matplotlib inline

In [3]:
import json
f = open('emojiclassify.json')
emojidata = json.load(f)
emojidata["emojis"]['😠']

'anger'

In [4]:
file = open("CodeMixedTestData.txt","r")
data = file.readlines()
data[0]

'<tweet>\n'

In [5]:
tweets = []
for line in data:
  line = line.split()
  if len(line) > 0:
    if line[0] == '<tweet>':
      readtweet = True
      tweet = []
      continue
    elif line[0] == '</tweet>':
      readtweet = False

    if readtweet == True:
      if line[0] == "<emotion>" :
        tweets.append({'text':tweet,"emotion":line[1]})
      else:
        tweet.append({'word':line[0],'lang':line[1]})

# Emoji and Emoticon Feature 

In [6]:
pip install emoji

     |████████████████████████████████| 170 kB 7.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=bd14b086a5d0106629ceb77a2227e2d03b2201c83494931d8700ad9853be13ff
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [7]:
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

In [8]:
all_features = []

def emoji_features(text):
  feature = [0,0,0]
  emojis_found = extract_emojis(text)
  for c in emojis_found:
    try:
      if emojidata["emojis"][c] == "happiness":
        feature[0] = feature[0] + 1
      elif emojidata["emojis"][c] == "sadness":
        feature[1] = feature[1] + 1
      elif emojidata["emojis"][c] == "anger":
        feature[2] = feature[2] + 1
    except:
      pass
  return feature

# Punctuation Feature

In [9]:
def punctuation_count(text):
  punctuation_num = 0
  for c in text:
    if c in ['!','?']:
      punctuation_num =  punctuation_num + 1

  return  punctuation_num

# Creating feature vector 


In [10]:
for tweet in tweets:
  othertext = ''
  for word in tweet['text']:
    if word['lang'] == 'OTH':
      othertext = othertext + word['word']

  feature = emoji_features(othertext)
  punctuation_num = punctuation_count(othertext)
  feature.append(punctuation_num)
  all_features.append(feature)

print(all_features)

[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [2, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 2], [0, 1, 0, 0], [0, 2, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 3, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]]


# Preprocessing



In [11]:
import re

In [12]:
tweetnum = 0
tweettextonly = []
for tweet in tweets:
  textonly = ''
  removewordlist = []
  for word in tweet['text']:
    if word['lang'] == 'OTH':
      removewordlist.append(word)
  for word in removewordlist:
    tweet['text'].remove(word)
  for word in tweet['text']:
    textonly = textonly + ' ' + word['word']
  tweettextonly.append(textonly)
print(tweettextonly)

[' I am happy loooool', ' I LOVE YOU', ' I am happy our mai khush hu', ' I am very happy', ' I am happy', ' I am happy', ' I am happy', ' I am happy', ' I am happy', ' I am Sad', ' I am Sad', ' I am Sad', ' I am Sad', ' I am angry', ' I AM ANGRY', ' I AM ANGRY', ' I AM ANGRY']


In [13]:
pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 15.9 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=53ae0f558319e763bff180575083a0f08b1a2624b417061a2f1a1bce99766d05
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [14]:
from googletrans import Translator
translator = Translator()
translator.translate("mai khush hu",dest='hi').text

'मै खुश हु'

In [15]:
for tweet in tweets:
  hintext = ''
  for word in tweet['text']:
    if word['lang'] == 'HIN':
      print(word['word'])
      hintext = hintext + ' ' + word['word']
  if hintext != '':
    devtext = translator.translate(hintext,dest='hi').text
    devtext = devtext.split()
    wordnum = 0
    for word in tweet['text']:
      if word['lang'] == 'HIN':
        word['word'] = devtext[wordnum]
        wordnum = wordnum + 1
tweets

our
mai
khush
hu


[{'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', 'word': 'am'},
   {'lang': 'ENG', 'word': 'happy'},
   {'lang': 'ENG', 'word': 'loooool'}]},
 {'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', 'word': 'LOVE'},
   {'lang': 'ENG', 'word': 'YOU'}]},
 {'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', 'word': 'am'},
   {'lang': 'ENG', 'word': 'happy'},
   {'lang': 'HIN', 'word': 'आवर'},
   {'lang': 'HIN', 'word': 'मै'},
   {'lang': 'HIN', 'word': 'खुश'},
   {'lang': 'HIN', 'word': 'हु'}]},
 {'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', 'word': 'am'},
   {'lang': 'ENG', 'word': 'very'},
   {'lang': 'ENG', 'word': 'happy'}]},
 {'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', 'word': 'am'},
   {'lang': 'ENG', 'word': 'happy'}]},
 {'emotion': 'Happiness',
  'text': [{'lang': 'ENG', 'word': 'I'},
   {'lang': 'ENG', '

# Uppercase


In [16]:
def uppercase_feature(text):
  splittweet = text.split()
  uppercase_words = 0
  for word in splittweet:   
    if word.upper() == word and len(word) > 1:
      uppercase_words = uppercase_words + 1
  return uppercase_words

In [17]:
tweetnum = 0
for tweet in tweettextonly:
  uppercase_words = uppercase_feature(tweet)
  all_features[tweetnum].append(uppercase_words)
  tweetnum = tweetnum + 1
  
print(all_features)

[[1, 0, 0, 0, 0], [1, 0, 0, 0, 2], [1, 0, 0, 0, 0], [2, 0, 0, 0, 0], [2, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 2, 0], [0, 1, 0, 0, 0], [0, 2, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 3, 0, 0], [0, 0, 1, 0, 2], [0, 0, 1, 0, 2], [0, 0, 1, 0, 2]]


# Repeated Charater Feature

In [18]:
def repeated_character_feature(text):
  splittweet = text.split()
  repeated_words = 0
  for word in splittweet:   
    first_word = ''
    second_word = ''
    for c in word:
      if c == first_word and first_word == second_word:
        repeated_words = repeated_words + 1
        break
      first_word = second_word
      second_word = c
  return repeated_words

In [19]:
tweetnum = 0
for tweet in tweettextonly:
  repeated_words = repeated_character_feature(tweet)
  all_features[tweetnum].append(repeated_words)
  tweetnum = tweetnum + 1

print(all_features)

[[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 2, 0], [1, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 2, 0, 0], [0, 1, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0], [0, 0, 1, 0, 2, 0], [0, 0, 1, 0, 2, 0], [0, 0, 1, 0, 2, 0]]


# Negation Words 

In [20]:
# importing english negated words
file = open("Negationwords.txt","r")
NegatedWords = file.read()
NegatedWordsList = NegatedWords.split()
NegatedWordsList

['never',
 'no',
 'nothing',
 'nowhere',
 'noone',
 'none',
 'not',
 'havent',
 'hasnt',
 'hadnt',
 'cant',
 'couldnt',
 'shouldnt',
 'wont',
 'wouldnt',
 'dont',
 'doesnt',
 'didnt',
 'isnt',
 'arent',
 'aint',
 'nahi',
 'nhi']

In [21]:
def negatedwords_feature(text):
  splittweet = text.split()
  negated_words = 0
  for word in splittweet:   
    if word.lower() in NegatedWordsList:
      negated_words = negated_words + 1
  return negated_words

In [22]:
tweetnum = 0
for tweet in tweettextonly:
  negated_words = negatedwords_feature(tweet)
  all_features[tweetnum].append(negated_words)
  tweetnum = tweetnum + 1

print(all_features)

[[1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 2, 0, 0], [1, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 2, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 0], [0, 0, 1, 0, 2, 0, 0], [0, 0, 1, 0, 2, 0, 0], [0, 0, 1, 0, 2, 0, 0]]


# Lexicon Emotion Feature

In [23]:
englexicon = pd.read_excel('NRC-Emotion-Lexicon.xlsx', sheet_name=None)
englexicon = englexicon['NRC-Lex-v0.92-word-translations']
englexicon.set_index("English (en)", inplace = True)
englexicon

,Hindi (hi),Positive,Negative,Anger,Disgust,Fear,Joy,Sadness,Surprise
English (en),,,,,,,,,
aback,अचंभे,0,0,0,0,0,0,0,0
abacus,अबेकस,0,0,0,0,0,0,0,0
abandon,छोड़ देना,0,1,0,0,1,0,1,0
abandoned,त्यागा हुआ,0,1,1,0,1,0,1,0
abandonment,संन्यास,0,1,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...
zone,क्षेत्र,0,0,0,0,0,0,0,0
zoo,चिड़ियाघर,0,0,0,0,0,0,0,0
zoological,प्राणि,0,0,0,0,0,0,0,0


In [24]:
hinlexicon = pd.read_excel('NRC-Emotion-LexiconHindi.xlsx', sheet_name=None)
hinlexicon = hinlexicon['NRC-Lex-v0.92-word-translations']
hinlexicon.set_index("Hindi (hi)",inplace = True)

In [25]:
def lexicon_emotion_feature(text):
  tweetemotion = [0,0,0]
  for word in text['text']:
    try:
      if word['lang'] == 'ENG':
        wordemotion = englexicon.loc[word['word'].lower()]
      elif word['lang'] == 'HIN':
        wordemotion = hinlexicon.loc[word['word'].lower()]
      tweetemotion[0] = tweetemotion[0] + wordemotion['Joy']
      tweetemotion[1] = tweetemotion[1] + wordemotion['Sadness']
      tweetemotion[2] = tweetemotion[2] + wordemotion['Anger']
    except:
      pass
  return tweetemotion

In [26]:
tweetnum = 0
for tweet in tweets:
  tweetemotion = lexicon_emotion_feature(tweet)
  all_features[tweetnum] = all_features[tweetnum] + tweetemotion
  tweetnum = tweetnum + 1

print(all_features)

[[1, 0, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 0, 2, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [2, 0, 0, 0, 0, 0, 0, 1, 0, 0], [2, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 2, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1]]


# Intensifier Feature

In [27]:
# importing english intensifiers
file = open("IntensifierList.txt","r")
Intensifiers = file.read()
IntensifiersList = Intensifiers.split()

In [28]:
def Intensifiers_feature(text):
  splittweet = text.split()
  Intensifier_words = 0
  for word in splittweet:   
    if word.lower() in IntensifiersList:
      Intensifier_words = Intensifier_words + 1
  return Intensifier_words

In [29]:
tweetnum = 0
for tweet in tweettextonly:
  Intensifier_words = Intensifiers_feature(tweet)
  all_features[tweetnum].append(Intensifier_words)
  tweetnum = tweetnum + 1

print(all_features)

[[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0]]


# Text Tagging

In [30]:
tag = []
for tweet in tweets:
  tag.append(tweet['emotion'])
print(tag)

['Happiness', 'Happiness', 'Happiness', 'Happiness', 'Happiness', 'Happiness', 'Happiness', 'Happiness', 'Happiness', 'Sadness', 'Sadness', 'Sadness', 'Sadness', 'Anger', 'Anger', 'Anger', 'Anger']


# Tagging and Predicting

In [32]:
X = all_features
y = tag
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X, y)

In [33]:
poly_pred = poly.predict([[0, 5, 0, 0, 0, 1, 0, 1, 0, 0, 0]])
print(poly_pred)

['Sadness']
